Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock, Germany

Tutorial Digital Signal Processing (Course #24505),
**Uniform Quantization, Dithering, Noiseshaping**,
Winter Semester 2019/20

Feel free to contact lecturer frank.schultz@uni-rostock.de

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

WIP...

# Fundamentals

## Packages / Functions

We import the required packages first.

In [ ]:
# most common used packages for DSP, have a look into other scipy submodules
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal
#from scipy import fftpack
#from scipy import stats

#audio write and play stuff
import soundfile as sf
import IPython.display as ipd

def my_xcorr2(x, y, scaleopt='none'):
    N = len(x)
    M = len(y)
    kappa = np.arange(0, N+M-1) - (M-1)
    ccf = signal.correlate(x, y, mode='full', method='auto')
    if N == M:
        if scaleopt == 'none' or scaleopt == 'raw':
            ccf /= 1
        elif scaleopt == 'biased' or scaleopt == 'bias':
            ccf /= N
        elif scaleopt == 'unbiased' or scaleopt == 'unbias':
            ccf /= (N - np.abs(kappa))
        elif scaleopt == 'coeff' or scaleopt == 'normalized':
            ccf /= np.sqrt(np.sum(x**2) * np.sum(y**2))
        else:
            print('scaleopt unknown: we leave output unnormalized')
    return kappa, ccf

### Midtread Quantization Characteristic Curve

Next, we should implement and validate the saturated uniform midtread quantizer.
We do this with two approaches. `uniform_midtread_quantizer` follows the lecture using `floor` operation, `my_quant` as an alternative approach uses `round` operation.
Both approaches yield the same results besides a slight detail: `uniform_midtread_quantizer` always exhibits **even** number of quantization steps.

Detailed analysis for `my_quant` would be:

- it always saturates $x<-1$ towards $x_q = -1$
- in the case of an **odd** number of quantization steps $Q$, it saturates $x>+1$ towards $x_q = +1$. The quantization step size is $\Delta Q = \frac{2}{Q}$.
- In the case of an **even** number of quantization steps $Q$, it saturates $x>\frac{Q - 1 - \frac{Q}{2}}{\frac{Q}{2}}$ towards $x_q = \frac{Q - 1 - \frac{Q}{2}}{\frac{Q}{2}}$. The quantization step size is $\Delta Q = \frac{2}{Q-1}$.

The last case is practically used for virtually all analog/digital (AD) and digital/analog (DA) converters: When additionally

\begin{equation}
\log_2(Q)\in\mathbb{N}
\end{equation}

holds, it is meaningful to code the $Q$ possible quantization steps with bits.
Said differently: the number range convention for AD and DA converters is 

\begin{equation}
-1\leq x \leq 1-2^{-(B-1)}
\end{equation}

using 

\begin{equation}
Q=2^B
\end{equation}

quantization steps, where $B\in\mathbb{Z}$ denotes the number of bits.
Values of $x$ outside this range will be saturated to their minimum and maximum quantization values in the quantization process.

For example, $B = 16$ bit is used for CD audio quality.
Then we get the following quantities.

In [ ]:
B = 16
Q = 2**B
xqmax = 1-2**(-(B-1))
# or more general for even Q
xqmax = (Q-1-Q/2)/(Q/2)
deltaQ = 2/Q  # for even Q only
print(' B = %d bits\n quantization steps Q = %d\n quantization step size %e' %
      (B, Q, deltaQ))
print(' smallest quantization value xqmin=-1')
print(' largest quantization value xqmax=%16.15f' % xqmax)
# B = 16 bits
# quantization steps Q = 65536
# quantization step size 3.051758e-05
# smallest quantization value xqmin=-1
# largest quantization value xqmax=0.999969482421875

In [ ]:
def uniform_midtread_quantizer(x, deltaQ):
    r"""uniform_midtread_quantizer from the lecture:
    https://github.com/spatialaudio/digital-signal-processing-lecture/blob/master/quantization/linear_uniform_quantization_error.ipynb
    commit: b00e23e
    note: we renamed the second input to deltaQ, since this is what the variable
    actually represents, i.e. the quantization step size
    """
    # limiter
    x = np.copy(x)
    idx = np.where(x <= -1)
    x[idx] = -1
    idx = np.where(x > 1 - deltaQ)
    x[idx] = 1 - deltaQ
    # linear uniform quantization
    xQ = deltaQ * np.floor(x/deltaQ + 1/2)
    return xQ

In [ ]:
def my_quant(x, Q):
    r"""Saturated uniform midtread quantizer

    x  input signal
    Q  number of quantization steps
    xq quantized output signal

    Note: for even Q in order to retain midtread characteristics,
    we must omit one quantization step, either that for lowest or the highest
    amplitudes. Typically the highest signal amplitudes are saturated to
    the 'last' quantization step. Then, in the special case of log2(N)
    being an integer the quantization can be represented with bits.
    """
    tmp = Q//2  # integer div
    quant_steps = (np.arange(0, Q) - tmp) / tmp  # we don't use this

    # forward quantization, round() and inverse quantization
    xq = np.round(x*tmp) / tmp
    # always saturate to -1
    xq[xq < -1.] = -1.
    # saturate to ((Q-1) - (Q\2)) / (Q\2), note that \ is integer div
    tmp2 = ((Q-1) - tmp) / tmp  # for odd N this always yields 1
    xq[xq > tmp2] = tmp2
    return xq

In [ ]:
Q = 16  # number of quantization steps
deltaQ = 1/(Q//2)  # quantization step size
print('Q = %d, deltaQ=%e' % (Q, deltaQ))

x = np.arange(-1.25, +1.25, 1e-3)
xq = my_quant(x, Q)
xumq = uniform_midtread_quantizer(x, deltaQ)

plt.figure(figsize=(6, 6))
plt.plot(x, xumq, label='uniform_midtread_quantizer()')
plt.plot(x, xq, label='my_quant()')
plt.xticks(np.arange(-1, 1.25, 0.25))
plt.yticks(np.arange(-1, 1.25, 0.25))
plt.xlabel(r'input $x$')
plt.ylabel(r'output $x_q$')
plt.title('uniform saturated midtread quantization')
plt.axis('equal')
plt.legend()
plt.grid(True)

# Exercise 1: Uniform Saturated Midtread Characteristic Curve of Quantization

In [ ]:
def check_my_quant(Q):
    N = 1e4
    x = 2*np.arange(0, N)/N - 1
    xq = my_quant(x, Q)
    e = xq - x

    plt.plot(x, x, color='C2', label=r'$x[k]$')
    plt.plot(x, xq, color='C3', label=r'$x_q[k]$')
    plt.plot(x, e, color='C0', label=r'$e[k] = x_q[k] - x[k]$')
    plt.xticks(np.arange(-1, 1.25, 0.25))
    plt.yticks(np.arange(-1, 1.25, 0.25))
    plt.xlabel('input')
    plt.ylabel('output')
    if np.mod(Q, 2) == 0:
        s = ' saturated '
    else:
        s = ' '
    plt.title('uniform'+s+'midtread quantization with Q='+str(Q)+' steps')
    plt.axis('equal')
    plt.legend(loc='upper left')
    plt.grid(True)

In [ ]:
Q = 7  # number of quantization steps
deltaQ = 1 / (Q//2)  # general rule
deltaQ = 2 / (Q-1)  # for odd Q only
check_my_quant(Q)
print('Q = %d, deltaQ=%f' % (Q, deltaQ))
# Q = 7, deltaQ=0.333333

In [ ]:
Q = 8  # number of quantization steps
deltaQ = 1 / (Q//2)  # general rule
deltaQ = 2 / Q  # for even Q only
check_my_quant(Q)
print('Q = %d, deltaQ=%f' % (Q, deltaQ))
# Q = 8, deltaQ=0.250000

# Exercise 2: Quantization and Signal-to-Noise Ratio

In [ ]:
def check_quant_SNR(x, dBoffset, title):
    print(np.std(x), np.var(x), np.mean(x))
    Bmax = 24
    SNR = np.zeros(Bmax+1)
    SNR_ideal = np.zeros(Bmax+1)

    for B in range(1, Bmax+1):  # start at 1, since zero Q is not meaningful
        xq = my_quant(x, 2**B)
        SNR[B] = 10*np.log10(np.var(x) / np.var(xq-x))
        SNR_ideal[B] = B*20*np.log10(2) + dBoffset  # 6dB/bit + offset rule

    plt.figure(figsize=(5, 5))
    plt.plot(SNR_ideal, 'o-', label='theoretical', lw=3)
    plt.plot(SNR, 'x-', label='simulation')
    plt.xticks(np.arange(0, 26, 2))
    plt.yticks(np.arange(0, 156, 12))
    plt.xlim(2, 24)
    plt.ylim(6, 148)
    plt.xlabel('number of bits')
    plt.ylabel('SNR / dB')
    plt.title(title)
    plt.legend()
    plt.grid(True)

In [ ]:
N = 10000
k = np.arange(0, N)

In [ ]:
np.random.seed(4)
x = np.random.rand(N)
x -= np.mean(x)
x *= np.sqrt(1/3) / np.std(x)
dBoffset = 0
check_quant_SNR(x, dBoffset, 'Uniform PDF')

In [ ]:
Omega = 2*np.pi * 997/44100  # use a rather odd ratio: e.g. in audio 997 Hz / 44100 Hz
sigma2 = 1/2
dBoffset = -10*np.log10(2 / 3)
x = np.sqrt(2*sigma2) * np.sin(Omega*k)
check_quant_SNR(x, dBoffset, 'Sine')

In [ ]:
np.random.seed(4)
x = np.random.randn(N)
x -= np.mean(x)
x *= np.sqrt(0.0471) / np.std(x)
dBoffset = -8.5  # from clipping propability 1e-5
check_quant_SNR(x, dBoffset, 'Normal PDF')

In [ ]:
np.random.seed(4)
x = np.random.laplace(size=N)
pClip = 1e-5  # clipping propability
sigma = -np.sqrt(2) / np.log(pClip)
x -= np.mean(x)
x *= sigma / np.std(x)
dBoffset = -13.5  # empircially found for pClip = 1e-5
check_quant_SNR(x, dBoffset, 'Laplace PDF')

# Exercise 3: Dithering

The discrete-time sine signal

- $x[k]=\Delta Q\cdot\sin(\frac{2\pi f_\text{sin}}{f_s}k)$,
- $0\leq k<10^5$ with 
- $f_\text{sin}=960\,\text{Hz}$
- and sampling frequency $f_s=48\,\text{kHz}$ 

shall be quantized with the $B=3$ Bit midtread quantizer shown below ($\Delta Q=\frac{1}{4}$, $|x_\text{max}|=1$).

In [ ]:
deltaQ = 1/4
Q = int(2/deltaQ)  # for even Q only
plt.figure(figsize=(4,4))
check_my_quant(Q)

Before quantizing $x[k]$, a dither noise signal $d[k]$ shall be added to $x[k]$ according figure below. 

<img src="Quantization_with_Dithering.png" alt="Quantization_with_Dithering" width="400"/>

This dither signal with small amplitudes aims at de-correlating the quantization error $e[k]$ from the quantized signal $x_q[k]$, which is especially important for small amplitudes of $x[k]$.
This technique is called **dithering**.
For $d[k]=0$ no dithering is applied.

Since the quantization error may be in the range $-\frac{\Delta Q}{2}\leq e[k]\leq \frac{\Delta Q}{2}$ (assuming uniform distribution), it appears reasonable to use a dither noise with a probability density function (PDF) of

\begin{equation}
p_\text{RECT}(d)=\frac{1}{\Delta Q}\,\text{rect}\left(\frac{d}{\Delta Q}\right),
\end{equation}

i.e. a **zero-mean, uniformly distributed noise** with maximum amplitude $|d[k]|=\frac{\Delta Q}{2}$.
It can be shown that this dither noise improves the quality of the quantized signal.
However, there is still a noise modulation (i.e. a too high correlation between $x_q[k]$ and $e[k]$) that depends on the amplitude of the input signal.

The noise modulation can be almost completely eliminated with a **zero-mean noise** signal exhibiting a **symmetric triangular PDF**

\begin{equation}
p_\text{TRI}(d)=\frac{1}{\Delta Q}\,\text{tri}\left(\frac{d}{\Delta Q}\right)
\end{equation}

with maximum amplitude $|d[k]|=Q$.
By doing so, an almost ideal decorrelation between $x_q[k]$ and $e[k]$ is realized.
In audio, this technique is called TPDF-Dithering (Triangular Probability Density Function Dithering) and can be applied in the mastering process of audio material that is to be distributed e.g. on a CD or via streaming.

# Task 

To get an impression on how dithering may be implemented and what quantized signals sound like, the following exercises shall be performed.

- Generate the sine signal $x[k]$ defined above.

- Generate the dither noise $d_\text{RECT}[k]$ according to the PDF $p_\text{RECT}(d)$. Check the resulting amplitude and distribution carefully. The length of $d_\text{RECT}[k]$ and $x[k]$ must be equal.

- Generate the dither noise $d_\text{TRI}[k]$ according to the PDF $p_\text{TRI}(d)$. Check the resulting amplitude and distribution carefully. The length of $d_\text{TRI}[k]$ and $x[k]$ must be equal.

- Add each dither noise $d_\text{RECT}[k]$ and $d_\text{TRI}[k]$ individually to $x[k]$. Together with the case of no dithering we now have three signals to be quantized.

- Quantize these signals individually using `my_quant(x,Q)` with $Q=2^B=2^3=8$ quantization steps.

- Plot the midtread characteristic curve.

- Plot the histogram of the dither noises as estimate of its PDF.

- Plot the sine signal, the dithered signal, the quantized signal and the quantization error signal in one diagram for all three cases.

- Calculate and plot the CCF of the signals $x_q[k]$ and $e[k]$ for all three cases.

- Interpret the obtained graphics.

- For each case, render WAV files from $x[k]$, $x[k]+d[k]$, $x_q[k]$ und $e[k]$ and listen to them. **Be careful! Do not harm your ears!** Pay special attention to the sound of the quantization error, how it is correlated with the quantized signal and how loud it appears.

- We are now ready to quantize and listen to signals whose amplitude is even below the quantization step size $\Delta Q$. For that, use the sine signal this time with amplitude $\frac{\Delta Q}{8}$ and dithering with RECT and TRI dither noise. Compare with the results from above case.

## Solution

The task asks for repeated steps.
This is perfectly handled by a little function that solves the repeating subtasks.

In [ ]:
def check_dithering(x, dither, Q, case):

    deltaQ = 1/(Q//2)  # general rule

    # dither noise
    pdf, edges = np.histogram(dither, bins='auto', density=True)
    xd = x + dither

    # quantization
    xq = my_quant(xd, Q)
    e = xq-x

    # write wavs
    sf.write(file='x_'+case+'.wav', data=x,
             samplerate=48000, subtype='PCM_24')
    sf.write(file='xd_'+case+'.wav', data=xd,
             samplerate=48000, subtype='PCM_24')
    sf.write(file='xq_'+case+'.wav', data=xq,
             samplerate=48000, subtype='PCM_24')
    sf.write(file='e_'+case+'.wav', data=e,
             samplerate=48000, subtype='PCM_24')

    # CCF
    kappa, ccf = my_xcorr2(xq, e, scaleopt='biased')

    # plot dither noise PDF estimate as histogram
    plt.figure(figsize=(12, 3))
    plt.subplot(1, 2, 1)
    plt.plot(edges[:-1], pdf, 'o-', ms=5, label='histogram')
    plt.xticks(np.arange(-deltaQ*3/2, deltaQ*3/2+deltaQ/2, deltaQ/2))
    plt.xlim(-deltaQ*3/2, +deltaQ*3/2)
    plt.ylim(-0.1, (1/deltaQ)*1.1)
    plt.grid(True)
    plt.xlabel(r'$\theta$')
    plt.ylabel(r'$\hat{p}(\theta)$')
    plt.title('PDF estimate of dither noise')

    # plot midtread characterstic curve
    plt.subplot(1, 2, 2)
    check_my_quant(Q)

    # plot signals
    plt.figure(figsize=(12, 3))
    plt.subplot(1, 2, 1)
    plt.plot(k, x, color='C2', label=r'$x[k]$')
    plt.plot(k, xd,  color='C1', label=r'$x[k] + dither[k]$')
    plt.plot(k, xq,  color='C3', label=r'$x_q[k]$')
    plt.plot(k, e,  color='C0', label=r'$e[k] = x_q[k] - x[k]$')
    plt.plot(k, k*0+deltaQ, ':k', label=r'$\Delta Q$')
    plt.xlabel('k')
    plt.title('signals')
    plt.xticks(np.arange(0, 175, 25))
    plt.yticks(np.arange(-2*deltaQ, 2*deltaQ+deltaQ/2, deltaQ/2))
    plt.xlim(0, 150)
    plt.ylim(-2*deltaQ, 2*deltaQ)
    plt.legend(loc='lower left')
    plt.grid(True)

    # plot CCF
    plt.subplot(1, 2, 2)
    plt.plot(kappa, ccf)
    plt.xlabel(r'$\kappa$')
    plt.ylabel(r'$\varphi_{xq,e}[\kappa]$')
    plt.title('CCF betwen xq and e=xq-x')
    plt.xticks(np.arange(-100, 125, 25))
    plt.xlim(-100, 100)
    plt.grid(True)

In [ ]:
Q = 2**3
deltaQ = 1/(Q//2)
print('Q = %d, deltaQ=%e' % (Q, deltaQ))

N = 100000
fsin = 960
fs = 48000
k = np.arange(0, N)

x = deltaQ * np.sin(2*np.pi*fsin/fs*k)
#x = 1/8 * deltaQ * np.sin(2*np.pi*fsin/fs*k)

### No Dither Noise

In [ ]:
# no dither
check_dithering(x=x, dither=x*0, Q=Q, case='nodither')

In [ ]:
ipd.Audio('x_nodither.wav')

In [ ]:
ipd.Audio('xd_nodither.wav')

In [ ]:
ipd.Audio('xq_nodither.wav')

In [ ]:
ipd.Audio('e_nodither.wav')

### Uniform PDF Dither Noise

In [ ]:
# uniform dither with max amplitude of deltaQ/2 
np.random.seed(1)
dither_uni = (np.random.rand(N) - 0.5) * 2 * deltaQ/2

check_dithering(x=x, dither=dither_uni, Q=Q, case='unidither')

In [ ]:
ipd.Audio('x_unidither.wav')

In [ ]:
ipd.Audio('xd_unidither.wav')

In [ ]:
ipd.Audio('xq_unidither.wav')

In [ ]:
ipd.Audio('e_unidither.wav')

### Triangular PDF Dither Noise

In [ ]:
np.random.seed(1)
# uniform PDF for amplitudes -1...+1:
dither_uni1 = (np.random.rand(N) - 0.5) * 2
dither_uni2 = (np.random.rand(N) - 0.5) * 2
# triangular PDF with max amplitude of deltaQ
dither_tri = (dither_uni1 + dither_uni2) * deltaQ/2

check_dithering(x=x, dither=dither_tri, Q=Q, case='tridither')

In [ ]:
ipd.Audio('x_tridither.wav')

In [ ]:
ipd.Audio('xd_tridither.wav')

In [ ]:
ipd.Audio('xq_tridither.wav')

In [ ]:
ipd.Audio('e_tridither.wav')

# **Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises